In [111]:
import random
from cells import Cell
from bp_class import *
from short_DNA_region_class import *
from chromosome_class import *
from sister_chromatids import *
import dna_functions

m = dna_functions.random_bp()
n = dna_functions.random_bp()
print(m == m)

#random.choice(["a", "c", "g", "t"])



True
